# Creating Organizer Topics using the API
## Introduction
Algorithmically creating topics using Seeq's REST API can be done using any programming language.  This tutorial will walk through creating these object using Python 2.7 and Seeq's SDK.

The tutorial will show how to create a new Organizer containing a new Topic and modifying the text shown in the topic.  Once this has been accomplished, I recommend that you check out the _Embedding Metrics and Trends using the API_ tutorial to see how to create and include Seeq content.

Note that doing these kinds of operations through the API circumvents all of the front-end's safety checks.  It is **highly recommended** that you develop on a local machine and test all code thoroughly **before using your code on an active server**. You should probably also **back that server up**. As an example of what can go wrong, forgetting the `"`'s in the `workbook_input.data` JSON will render the server unreachable by all...and potentially unfixable!


## Creating an Organizer Topic
Organizer Topics, and Workbench Analyses are handled similarly by Seeq's backend.  Creating and manipulating them is done through the same API endpoint, namely the Workbooks endpoint.  The basic process for creating an Organizer Topic is similar to that for creating a Workbench Analysis, with the addition of creating a document to show in the Topic using the Annotations endpoint.

Creating the Organizer Topic is done in the following steps:
1. Create a new Organizer Topic
1. Update the state of the Organizer Topic as blank. 
1. Create a new Document in the Organizer.
1. Update the "Workstep" for the Document, initializing things like Date Ranges.
1. Create content for the Document.
1. Update the content in the Document.

### Setting up the Environment

In [1]:
# import the Seeq SDK for communicating with Seeq (I installed this from the egg distributed with Seeq and easy_install)
import seeq_sdk as sdk
# import Jinja2 for creating html templates to post in the document
import jinja2 as ji
# import uuid so that we can create UUID's (aka GUIDs) for our objects
from uuid import uuid4
# import json so that to parse objects
import json
# import datetime for working with times
from datetime import datetime as dt
# import copy so that we can use templates
import copy

### Connecting to Seeq
This is the basic stuff.  Note that these are dummy credentials for a local installation which will not work on your machine or in the cloud.

In [2]:
# create the API client
api_client = sdk.ApiClient("http://localhost:34216/api", None, None)

# get the clent authoization and login
auth_input = sdk.AuthInputV1(username='seth.gilchrist@seeq.com', password='testpass')
auth_api = sdk.AuthApi(api_client)
auth_output = auth_api.login(body=auth_input)

### Creating a new Organizer Topic
To create an Organizer Topic, we use the same API call that we would to make a new Workbench Analysis, except that we pass the `"isReportBinder": true` flag in the JSON body.

Remember that JSON requires double quotes (`"`) to encapsulate strings.  You can always visit the API help in Seeq (Hamburger Menu -> API Reference) to find out what properties are required.

In [3]:
topic_input = sdk.WorkbookInputV1()
topic_input.name = 'My First API Created Organizer'
topic_input.data = '{"isReportBinder":true}'

workbook_api = sdk.WorkbooksApi(api_client)
topic_output = workbook_api.create_workbook(body=topic_input)
print(topic_output)

{'access_level': u'FULL_CONTROL',
 'ancestors': [],
 'created_at': u'2019-03-05T22:12:22.114Z',
 'data': u'{"isReportBinder":true}',
 'description': None,
 'href': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809',
 'id': u'A9BD8616-633C-4755-8527-9EBEBD420809',
 'is_archived': False,
 'marked_as_favorite': False,
 'name': u'My First API Created Organizer',
 'owner': {'href': u'/users/7311CCDB-8572-4BB9-962A-E91D97095B8F',
           'id': u'7311CCDB-8572-4BB9-962A-E91D97095B8F',
           'is_archived': False,
           'name': u'Seth Gilchrist',
           'type': u'User'},
 'parent_folder_id': None,
 'status_message': None,
 'type': u'Workbook',
 'updated_at': u'2019-03-05T22:12:22.114Z',
 'worksheets': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809/worksheets'}


Once the Organizer Topic has been created, we have to update it and give it a state property using the Items API endpoint.  In this property, we will initialize a "store", which is Seeq terminology for the state of an Analysis or Topic, it is the content of a so-called "Workstep".  Later, we will update the store again through when setting the date ranges in our Organizer Topic. Without setting this property, the workbook will be unreachable through Seeq.

The `version` value is important when passing stores to Seeq. They are used to determine upgrade procedure as the software grows.  They are often the only _required_ property when defining stores through the API. The API will not reject a call without one, but it is important to include them.

In [4]:
topic_property_input = sdk.PropertyInputV1()
topic_property_input.unit_of_measure = 'string'
topic_property_input.value = '{"version":1,"state":{"stores":{}}}'

items_api = sdk.ItemsApi(api_client)
topic_state_output = items_api.set_property(id=topic_output.id,
                                            property_name='workbookState',
                                            body=topic_property_input)
print(topic_state_output)

{'href': u'/items/A9BD8616-633C-4755-8527-9EBEBD420809/properties/workbookState',
 'status_message': None}


### Creating a new Document
In this example I will create a new Topic Document, which is done through the Workbook/Worksheets API.  If you have a "template" document, you can duplicate that document by passing its ID in the `branchFrom` property of the `WorksheetInputV1()` object.  To create a new one, you can leave it unspecified or, as I've done here, assign it `None`.

In [5]:
document_input = sdk.WorksheetInputV1()
document_input.name = 'API Created Document 1'
document_input.branchFrom = None

document_output = workbook_api.create_worksheet(workbook_id=topic_output.id,
                                                body=document_input)
print(document_output)

{'created_at': u'2019-03-05T22:12:22.202Z',
 'description': None,
 'href': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809/worksheets/692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
 'id': u'692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
 'is_archived': False,
 'name': u'API Created Document 1',
 'report': None,
 'status_message': None,
 'type': u'Worksheet',
 'updated_at': u'2019-03-05T22:12:22.202Z',
 'workstep': None}


### Create a Workstep
Now that we have a Organizer Topic and document, we need to create a Workstep that will tell Seeq the configuration of the Document. Data about the Workstep configuration are stored in a dataStore.  This isn't the text of the document, but the state of other aspects, such as date ranges or, in the case of a Workbench Analysis, the display range and line colors.

The dataStore is a collection of JSON objects. We can specify any objects that we want to defined and the rest will be populated with default values.  For this example, I am going to define two date ranges that I want stored in the Workstep.

To create the date ranges, I'm going to define a function that takes in lists of start times and end times and populates an empty dateVariable dataStore.  The structure of a dataStore can always be found by looking at the `data` element of a Workstep retrieved through the API reference.

I'll start with the doc string and some constants, including a template of the dateVariable dataStore taken from the Workstep of any Organizer Topic or Workbench Analysis without a defined date range. Next I will move through the input start and end times and populate a new dateVariable dataStore for each pair.  dateVariables are passed in a list in the sqReportStore, hence the final wrapping in `sqReportStore`.  Note the double squiggly brackets (`{{}}`) for escaping when using `format` on JSON objects.  I also return a dictionary with the data range info that is a bit easier to navigate for reference when embedding the Metrics.

In [6]:
def date_variables_create(start_date, end_date, name=None):
    """
    Creates date variables for a report store.  Input lists of start and end dates to 
    generate a store with multiple date ranges
    Dates should be in ISO 8601 format: yyyy-MM-DDThh:mm:ss.ffffff.
    Note: nanoseconds are not supported, the highest precision in microseconds.
    Note: Timezone is not supported.  Convert to UTC before inputting
    
    :param start_date:  list of start times as ISO 8601 strings
    :param end_date: list of end times as ISO 8601 strings
    :param name: (optional) list of string names for the date ranges. Defaults to "API Created Date Range #"
    :return: the dateVariables with the sqReportStore wrapper
    """

    ms_per_second = 1000
    time_format = '%Y-%m-%dT%H:%M:%S.%f'
    epoch_start = dt(1970, 1, 1)
    default_name = "API Created Date Range {0}"
    date_variable_template = \
        '{\
            "id":"uuid",\
            "name":"name",\
            "range":\
            {\
                "start":0,\
                "end":0\
            },\
            "auto":\
            {\
                "enabled":false,\
                "duration":0,\
                "offset":\
                {\
                    "value":0,\
                    "units":"min"\
                },\
                "offsetDirection":"past",\
                "rate":\
                {\
                    "value":5,\
                    "units":"min"\
                }\
            },\
            "condition":\
            {\
                "strategy":"closestTo",\
                "reference":"end",\
                "offset":1\
            }\
        }'

    date_variable_json = json.loads(date_variable_template)
    
    # If only single start and end dates were passed, wrap them in a list for iteration
    if not isinstance(start_date, list):
        start_date = [start_date]

    if not isinstance(end_date, list):
        end_date = [end_date]
    
    # assign default names if none are given
    if name is None:
        name = [default_name.format(i+1) for i in range(len(start_date))]
    elif not isinstance(name, list):
        name = [name]
    
    assert(len(start_date) == len(name) and len(start_date) == len(end_date))

    date_variables = ''
    date_range_info = {}

    for s, e, n in zip(start_date, end_date, name):
        start_ms_timestamp = \
            (dt.strptime(s, time_format) - epoch_start).total_seconds() * ms_per_second
        end_ms_timestamp =  \
            (dt.strptime(e, time_format) - epoch_start).total_seconds() * ms_per_second
        ID = uuid4()
        current_date_variable = copy.copy(date_variable_json)
        current_date_variable['id'] = str(ID)
        current_date_variable['name'] = n
        current_date_variable['range']['start'] = start_ms_timestamp
        current_date_variable['range']['end'] = end_ms_timestamp        
        # append to the list
        date_variables += (json.dumps(current_date_variable) + ',')
        # add to the info by name
        date_range_info[n] = {'id': ID, 'start_ms': start_ms_timestamp, 'end_ms': end_ms_timestamp,
                             'start_iso': s, 'end_iso': e}

    # remove the trailing comma on the last one
    date_variables = date_variables[:-1]
    
    # return the ReportStore
    return '"sqReportStore":{{"dateVariables":[{0}]}}'.format(date_variables), date_range_info

Now that I can create sqReportStores, I'll create one with two date ranges.  Note that while the Seeq Backend works in nanoseconds, the Frontend works in microseconds, a limitation inherited from Javascript.

In [7]:
starts = ['2018-12-12T00:00:00.000000', '2019-01-05T13:32:18.934002']
ends = ['2018-12-15T00:00:00.000000', '2019-01-05T15:32:18.934002']

sq_report_store = date_variables_create(starts, ends)
print(sq_report_store)

('"sqReportStore":{"dateVariables":[{"auto": {"duration": 0, "rate": {"units": "min", "value": 5}, "offsetDirection": "past", "enabled": false, "offset": {"units": "min", "value": 0}}, "range": {"start": 1544572800000.0, "end": 1544832000000.0}, "id": "95f06f64-1117-4eb2-b3cb-f47aa413be45", "condition": {"offset": 1, "reference": "end", "strategy": "closestTo"}, "name": "API Created Date Range 1"},{"auto": {"duration": 0, "rate": {"units": "min", "value": 5}, "offsetDirection": "past", "enabled": false, "offset": {"units": "min", "value": 0}}, "range": {"start": 1546695138934.002, "end": 1546702338934.002}, "id": "64efd6ad-3831-48b9-9194-bcb28a985756", "condition": {"offset": 1, "reference": "end", "strategy": "closestTo"}, "name": "API Created Date Range 2"}]}', {'API Created Date Range 2': {'end_iso': '2019-01-05T15:32:18.934002', 'start_iso': '2019-01-05T13:32:18.934002', 'end_ms': 1546702338934.002, 'id': UUID('64efd6ad-3831-48b9-9194-bcb28a985756'), 'start_ms': 1546695138934.002},

Now that we have our desired starting point stores, I'll create a new Workstep, defining only these stores.  The version number was acquired from the empty dataStore I used as my template.  Again note the double curly braces (`{{}}`) since I'm formating using `format`.

In [8]:
topic_workstep_input = sdk.WorkstepInputV1()
topic_workstep_input.data = '{{"version":23,"state":{{"stores":{{{0}}}}}}}'.format(sq_report_store[0])

topic_workstep_output = workbook_api.create_workstep(workbook_id=topic_output.id,
                                                     worksheet_id=document_output.id,
                                                     body=topic_workstep_input)
print(topic_workstep_output)

{'created_at': u'2019-03-05T22:12:22.311Z',
 'data': u'{"version":23,"state":{"stores":{"sqReportStore":{"dateVariables":[{"auto": {"duration": 0, "rate": {"units": "min", "value": 5}, "offsetDirection": "past", "enabled": false, "offset": {"units": "min", "value": 0}}, "range": {"start": 1544572800000.0, "end": 1544832000000.0}, "id": "95f06f64-1117-4eb2-b3cb-f47aa413be45", "condition": {"offset": 1, "reference": "end", "strategy": "closestTo"}, "name": "API Created Date Range 1"},{"auto": {"duration": 0, "rate": {"units": "min", "value": 5}, "offsetDirection": "past", "enabled": false, "offset": {"units": "min", "value": 0}}, "range": {"start": 1546695138934.002, "end": 1546702338934.002}, "id": "64efd6ad-3831-48b9-9194-bcb28a985756", "condition": {"offset": 1, "reference": "end", "strategy": "closestTo"}, "name": "API Created Date Range 2"}]}}}}',
 'description': None,
 'href': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809/worksheets/692E9DBC-8C27-40D3-9A69-D3EC50CC001C/workstep

### Creating Content
Next step is to create some content, which we will do through the Annotations API.  The annotation we generate will hold the html that is our document content.  Annotations are used in Seeq for both Topics and Journals, and this one will be created with the type specified as `Report` to indicate it's a Topic Annotation.  The default is `Journal`.

The `interests` list should only have the Worksheet ID of the Topic document.  This tells Seeq to display this Annotation in that document.

In [9]:
annotation_input = sdk.AnnotationInputV1()

annotations_interested_input = sdk.AnnotationInterestInputV1()
annotations_interested_input.interest_id = document_output.id
annotation_input.interests = [annotations_interested_input]

annotation_input.type = "Report"
annotation_input.name = "Unnammed"

annotations_api = sdk.AnnotationsApi(api_client)
annotation_output = annotations_api.create_annotation(body=annotation_input)
print(annotation_output)

{'access_level': u'FULL_CONTROL',
 'backups': [],
 'created_at': u'2019-03-05T22:12:22.345Z',
 'created_by': {'href': u'/users/7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'id': u'7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'is_archived': False,
                'name': u'Seth Gilchrist',
                'type': u'User'},
 'description': None,
 'discoverable': False,
 'document': None,
 'href': u'/annotations/F9788407-7C3A-478B-9F2F-808487BC50D7',
 'id': u'F9788407-7C3A-478B-9F2F-808487BC50D7',
 'interests': [{'capsule': None,
                'item': {'href': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809/worksheets/692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
                         'id': u'692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
                         'is_archived': False,
                         'name': u'API Created Document 1',
                         'type': u'Worksheet'}}],
 'is_archived': False,
 'name': u'Unnammed',
 'published_at': None,
 'replies': [],
 

Now that we have the Annotation ID, lets load it with some content.  I'm going to make an html template using Jinja2.  Normally the html template would be specified in an external file, but for this demo, I'm going to define it in-line.

In [10]:
html_template = r'\
<p>\
    <a href="{{image_source}}">\
        <img src="{{image_source}}" style="width: {{width_percent}}%;\">\
    </a>\
    <br>\
    {{image_caption}}\
</p>'

jinja_environment = ji.Environment(loader=ji.BaseLoader).from_string(html_template)

html_document = jinja_environment.render(
    image_source=r"https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Glazed-Donut.jpg/1920px-Glazed-Donut.jpg",
    width_percent=20,
    image_caption="Seeqing for the Breakfast of Champions!!")

When we update the annotation, we must specify the name and re-specify the interested-in list, otherwise the connection between our document and the Annotation will be lost.

In [11]:
annotation_update_input = sdk.AnnotationInputV1()
annotation_update_input.name = "Unnamed"
annotation_update_input.type = "Report"
annotation_update_input.createdById = "Seth Gilchrist"
annotation_update_input.interests = [annotations_interested_input]
annotation_update_input.document = html_document

annotation_update_output = annotations_api.update_annotation(id=annotation_output.id, body=annotation_update_input)
print(annotation_update_output)

{'access_level': u'FULL_CONTROL',
 'backups': [],
 'created_at': u'2019-03-05T22:12:22.345Z',
 'created_by': {'href': u'/users/7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'id': u'7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'is_archived': False,
                'name': u'Seth Gilchrist',
                'type': u'User'},
 'description': None,
 'discoverable': False,
 'document': None,
 'href': u'/annotations/F9788407-7C3A-478B-9F2F-808487BC50D7',
 'id': u'F9788407-7C3A-478B-9F2F-808487BC50D7',
 'interests': [{'capsule': None,
                'item': {'href': u'/workbooks/A9BD8616-633C-4755-8527-9EBEBD420809/worksheets/692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
                         'id': u'692E9DBC-8C27-40D3-9A69-D3EC50CC001C',
                         'is_archived': False,
                         'name': u'API Created Document 1',
                         'type': u'Worksheet'}}],
 'is_archived': False,
 'name': u'Unnamed',
 'published_at': None,
 'replies': [],
 '

### Summary of Topic Creation
There you have it!  If you visit Seeq you'll find the document in place, with date ranges as specified, and a picture of everyone's favorite breakfast food.

You can embed many different kinds of content, including Seeq content, into documents using these techniques.  To learn how to include Seeq content in your organizer topic, take a look at the _Embedding Metrics and Trends using the API_ tutorial.

## Creating a Scorecards and Trends and Putting them into an Organizer Topic Document
Assuming that we have an Organizer Topic and a document that we can add content to, let's create a scorecard Metric and place it in the document.

The basic process for creating a Metric and putting it into a document is:
1. Create a Workbench Analysis Worksheet to display the Metric.
1. Create a Metric for our signal based on a condition and aggregation method.
1. Update the Worksheet with the metric information.
1. Generate a screenshot of the metric based on the Date Range.
1. Embed the screenshot into the Organizer Topic document.

### Creating a Workbench Analysis
The Workbench Analysis is created in a very similar manner to the Organizer Topic.  I will not go into detail on creating the analysis, as there is another how-to focused on that.  We will leave the Workstep uninitialized and create it after we have our Metric.

In [12]:
# create the Analysis
analysis_input = sdk.WorkbookInputV1()
analysis_input.name = "API Created Analysis"
analysis_output = workbook_api.create_workbook(body=analysis_input)

# create the worksheet
worksheet_input = sdk.WorksheetInputV1()
worksheet_input.name = "API Created Worksheet"
worksheet_output = workbook_api.create_worksheet(workbook_id=analysis_output.id, body=worksheet_input)

# set the initial workbook state
analysis_property_input = sdk.PropertyInputV1()
analysis_property_input.unit_of_measure = 'string'
analysis_property_input.value = '{"version":1,"state":{"stores":{}}}'
analysis_state_output = items_api.set_property(id=analysis_output.id,
                                               property_name='workbookState',
                                               body=analysis_property_input)
print("Workbook ID: {0}\nWorksheet ID: {1}".format(analysis_output.id, worksheet_output.id))

Workbook ID: 3039CCCF-FD4A-4A1F-83A8-280F9537AF50
Worksheet ID: BEFDCADE-3A31-49A3-A9FA-82DFAF85C94E


### Creating the Scorecard Metric
To create the metric, we need to have a signal ID and a condition ID against which to perform our calculations, and a aggregation function that we want to use to summarize the signal over the condition's capsules.  I will get my IDs from the Seeq interface, but they could come from API generated signals and conditions as well.  To replicate my example, you can use the IDs of the signals and conditions defined below.

Signal ID: ID of the example data `Example >> Cooling Tower 1 >> Area A >> Compressor Power`

Condition ID: ID of a value search on the signal above with a formula of

`$a.validValues().valueSearch(1day, isGreaterThan(25), 0min, isLessThanOrEqualTo(25), 0min)`

Once we have these IDs, we can use the Metrics API endpoint to create our metric.  I will be aggregating over the maximum value of the signal during the capsules (hence `aggregation_function = maxValue()`), but other aggregators are available.  To find the formula for the one you are interested in, create a scorecard with your target aggregator and examine the items properties in Seeq.

Note that `scoped_to` provides search scope for our metric, it's best to scope it to our Workbench Analysis so that it doesn't pollute other search locations. Also, instantiating `threasholds` to an empty list is required.

In [13]:
metric_input = sdk.ThresholdMetricInputV1()
metric_input.name = 'API Created Metric 1'
metric_input.measured_item = "5D9A8558-F592-4C3B-8361-D88FDD52F916"
metric_input.bounding_condition = "809E03A9-3949-4087-86D5-4B47FE3B2F8E"
metric_input.aggregation_function = "maxValue()"
metric_input.scoped_to = analysis_output.id
metric_input.thresholds = []

metric_api = sdk.MetricsApi(api_client)
metric_output = metric_api.create_threshold_metric(body=metric_input)
print(metric_output)

{'aggregation_function': u'maxValue()',
 'bounding_condition': {'ancestors': [],
                        'href': u'/conditions/809E03A9-3949-4087-86D5-4B47FE3B2F8E',
                        'id': u'809E03A9-3949-4087-86D5-4B47FE3B2F8E',
                        'is_archived': False,
                        'name': u'Value Search 1',
                        'type': u'CalculatedCondition'},
 'condition_thresholds': [],
 'data_id': None,
 'datasource_class': None,
 'datasource_id': None,
 'description': None,
 'display_item': {'href': u'/signals/3D6E6754-80A3-4F43-9837-8BDFA15C85AC',
                  'id': u'3D6E6754-80A3-4F43-9837-8BDFA15C85AC',
                  'is_archived': False,
                  'name': u'API Created Metric 1 Aggregation: maxValue()',
                  'type': u'CalculatedSignal'},
 'duration': None,
 'href': u'/metrics/31C58612-1A48-4BDF-9980-F199AF883AB4',
 'id': u'31C58612-1A48-4BDF-9980-F199AF883AB4',
 'is_archived': False,
 'measured_item': {'ancestors': [{'h

Now that the metric has been created, we will make a scorecard from it, which is done using a Workstep.  We will create a Workstep that shows the metric in a scorecard by applying appropriate values to two dateStores: the sqWorksheetStore, in which we set the `viewKey` to `SCORECARD` (an alternative being `TREND`) and an sqTrendMetricStore in which we tell Seeq how our Metric should be displayed.  You may want to play around with the `scorecardHeaders` to get them how you want.

In [14]:
worksheet_store = '\
{\
    "tabsets": {\
        "sidebar": 1\
    },\
    "viewKey": "SCORECARD",\
    "browsePanelCollapsed": false,\
    "displayWidth": 850,\
    "displayHeight": 575,\
    "resizeEnabled": true\
}'

trend_metric_template = '\
{\
    "scorecardHeaders": {\
        "type": "start",\
        "format": "lll"\
    },\
    "scorecardColumns": [\
        {\
            "type": "name",\
            "backgroundColor": "#ffffff"\
        }],\
    "dataStatus": "itemDataPresent",\
    "warningCount": 0,\
    "warningLogs": [],\
    "items": [\
        {\
            "axisAlign": "B",\
            "axisAutoScale": true,\
            "lane": 2,\
            "rightAxis": false,\
            "dashStyle": "Solid",\
            "lineWidth": 1,\
            "autoDisabled": false,\
            "axisVisibility": true,\
            "yAxisConfig": {\
              "min": -1.14,\
              "max": 1.14\
            },\
            "yAxisMin": -1,\
            "yAxisMax": 1,\
            "sampleDisplayOption": "line",\
            "id": "ID",\
            "name": "name",\
            "selected": false,\
            "color": "#9D248F",\
            "scorecardOrder": 0\
        }]\
}'

trend_metric_json = json.loads(trend_metric_template)
trend_metric_json['items'][0]['id'] = metric_output.id
trend_metric_json['items'][0]['name'] = metric_output.name

data_store = '{{"sqWorksheetStore": {0}, "sqTrendMetricStore": {1} }}'.format(worksheet_store, json.dumps(trend_metric_json))

analysis_workstep_input = sdk.WorkstepInputV1()
analysis_workstep_input.data = '{{"version":23,"state":{{"stores": {0} }} }}'.format(data_store)

analysis_workstep_output = workbook_api.create_workstep(workbook_id=analysis_output.id,
                                                        worksheet_id=worksheet_output.id,
                                                        body=analysis_workstep_input)
print(analysis_workstep_output)

{'created_at': u'2019-03-05T22:12:22.800Z',
 'data': u'{"version":23,"state":{"stores": {"sqWorksheetStore": {    "tabsets": {        "sidebar": 1    },    "viewKey": "SCORECARD",    "browsePanelCollapsed": false,    "displayWidth": 850,    "displayHeight": 575,    "resizeEnabled": true}, "sqTrendMetricStore": {"items": [{"lane": 2, "selected": false, "rightAxis": false, "name": "API Created Metric 1", "sampleDisplayOption": "line", "yAxisConfig": {"max": 1.14, "min": -1.14}, "axisAutoScale": true, "axisVisibility": true, "yAxisMax": 1, "autoDisabled": false, "color": "#9D248F", "scorecardOrder": 0, "axisAlign": "B", "lineWidth": 1, "id": "31C58612-1A48-4BDF-9980-F199AF883AB4", "yAxisMin": -1, "dashStyle": "Solid"}], "warningCount": 0, "scorecardColumns": [{"type": "name", "backgroundColor": "#ffffff"}], "dataStatus": "itemDataPresent", "warningLogs": [], "scorecardHeaders": {"type": "start", "format": "lll"}} } } }',
 'description': None,
 'href': u'/workbooks/3039CCCF-FD4A-4A1F-83A8-

You can now surf over to Seeq, open the Analysis and see the scorecard that you've created.

### Creating a Trend View
Creating scorecads is all well and good, but what about creating a trend view?  If we know what signals and conditions comprise a scorecard, then we can setup a trend showing them as well.

This can be done 100% algorithmically, but I would recommend setting up the trend just how you like it, grabbing a copy of the worksheet dataStore, and using that as a template.  From there, you can swap signals in and out, but most likely you'll be modifying only the date range of the trend view.

A typical workflow for embedding both a scorecard and trend in a topic would be the following:
* create the metric $\checkmark$
* create a Worksheet showing the metric $\checkmark$
* generate a screenshot of it (our next step)
* update the Worksheet's Workstep to show the trend
* generate a screenshot of that
Using this workflow you have one Worksheet that is used for both the scorecard and trend views.

While it's slightly out of sequence, I'm going to create the trend-view workstep now so that we can do the screenshots in quick succession.

Including signals in a trend view is done using the `sqTrendSeriesStore`, and including the condition is done using the `sqTrendCapsuleSetStore`.  Setting the view parameters happens in a few places in the dataStore, but the most important ones are in the `sqWorksheetStore`.  Let's define our parameters for these stores.

In [17]:
# We already have an sqWorksheetStore template, so we'll just update it.
worksheet_store_json = json.loads(worksheet_store)
worksheet_store_json['viewKey'] = "TREND"

# now for the other two
trend_series_store = '\
{\
    "items": [\
        {\
            "axisAlign": "A",\
            "axisAutoScale": true,\
            "lane": 1,\
            "rightAxis": false,\
            "dashStyle": "Solid",\
            "lineWidth": 1,\
            "autoDisabled": false,\
            "axisVisibility": true,\
            "yAxisConfig": {\
              "min": -1.2482005542500003,\
              "max": 37.00042770425\
            },\
            "yAxisMin": 0.0029228,\
            "yAxisMax": 35.74930435,\
            "sampleDisplayOption": "line",\
            "id": "",\
            "name": "",\
            "selected": false,\
            "color": "#E1498E",\
            "valueUnitOfMeasure": ""\
          }\
    ],\
        "editingId": null,\
        "previewSeriesDefinition": {}\
}'
trend_series_json = json.loads(trend_series_store)
# everything we need to know about the series we want to show is in metric_output, except for the unit of measure.
# However, the metric has the same UOM as the measured item, so we'll just use that.
trend_series_json['items'][0]['id'] = metric_output.measured_item.id
trend_series_json['items'][0]['name'] = metric_output.measured_item.name
trend_series_json['items'][0]['valueUnitOfMeasure'] = metric_output.value_unit_of_measure

trend_capsule_set_store = '\
{\
    "items": [\
        {\
            "autoDisabled": false,\
            "id": "ID",\
            "name": "",\
            "selected": false,\
            "color": "#00A2DD"\
         }\
    ]\
}'
trend_capsule_set_json = json.loads(trend_capsule_set_store)
# again, let's get all the info we need from metric_output
trend_capsule_set_json['items'][0]['id'] = metric_output.bounding_condition.id
trend_capsule_set_json['items'][0]['name'] = metric_output.bounding_condition.name

data_store = '\
{{\
    "version":23,\
    "state":{{\
        "stores":{{\
            "sqWorksheetStore": {0},\
            "sqTrendSeriesStore": {1},\
            "sqTrendCapsuleSetStore": {2}\
        }}\
    }}\
}}'.format(json.dumps(worksheet_store_json), json.dumps(trend_series_json), json.dumps(trend_capsule_set_json))

trend_workstep_input = sdk.WorkstepInputV1()
trend_workstep_input.data = data_store

Now that we have our trend view Workstep defined, let's move on to creating the screenshots.

### Obtaining a Screenshot for an Organizer Topic Document
The Organizer Topics use linked screenshots to display content.  This has the advantage of appearing as immutable content in the Topic document and using the existing Workbench Analysis rendering capabilities to generate the trends and tables.

Screenshots are created using the Jobs API endpoint.  The Jobs API setups the Worksheet containing the scorecard to a specific date range, followed by a "headless capture".  That is, it renders the web page in memory and saves a screenshot without ever displaying it.

The endpoint determines the date range to use for the screenshot from a capsule-producing formula, which is set in `range_formula`.  For scorecards, the `content_selector` must be set to `.screenshotSizeToContent` to ensure that the returned screenshot only shows the scorecard.  Note that when `content_selector` is set to `.screenshotSizeToContent`, the `height` and `width` parameters are ignored, even though they are required.

In the following screenshot, I will use one of the date ranges created in _Creating Organizer Topics using the API_ in which I stored the date range name, id, and start/end ISO8601 timestamps from an API created date range.  You will need these elements when creating and embedding the scorecard in the Topic document. I saved the date ranges in `date_range_info` previously.

In [49]:
# dictonary with structure {'id': str, 'start_ms': int, 'end_ms': int, 'start_iso': str, 'end_iso': str}
screenshot_date_range = sq_report_store[1]['API Created Date Range 1']

range_formula = 'capsule("{0}Z", "{1}Z")'.format(screenshot_date_range['start_iso'], screenshot_date_range['end_iso'])

jobs_api = sdk.JobsApi(api_client)
metric_screenshot_job_output = jobs_api.get_screenshot(
    worksheet_id=worksheet_output.id,
    workstep_id=analysis_workstep_output.id,
    content_selector='.screenshotSizeToContent',
    range_formula=range_formula,
    height=100, width=100)
print(metric_screenshot_job_output)

{'end': u'2018-12-15T00:00:00Z',
 'height': 67,
 'screenshot': u'/screenshots/_present_3039cccf-fd4a-4a1f-83a8-280f9537af50_befdcade-3a31-49a3-a9fa-82dfaf85c94e_b97155b3_1551829124603.png',
 'start': u'2018-12-12T00:00:00Z',
 'status_message': None,
 'width': 670}


The property `screenshot` of `screenshot_job_output` contains the internal URL of our screenshot.

Now that we have the metric screenshot, let's update the workstep and get the trend screenshot.  Note that I'm omitting `content_selector`, updating the `workstep_id`, and setting the `height` and `width` to values I pre-determined.  You'll need to save the height and width values as the jobs endpoint only returns them when it sets them.

In [46]:
trend_workstep_output = workbook_api.create_workstep(workbook_id=analysis_output.id,
                                                     worksheet_id=worksheet_output.id,
                                                     body=trend_workstep_input)
trend_width = 700
trend_height = int(700./(16./9.)) # make a "widescreen" screenshot
trend_screenshot_job_output = jobs_api.get_screenshot(
    worksheet_id=worksheet_output.id,
    workstep_id=trend_workstep_output.id,
    range_formula=range_formula,
    height=trend_height, width=trend_width)
print(trend_screenshot_job_output)

{'end': u'2018-12-15T00:00:00Z',
 'height': None,
 'screenshot': u'/screenshots/_present_3039cccf-fd4a-4a1f-83a8-280f9537af50_befdcade-3a31-49a3-a9fa-82dfaf85c94e_7e3349cb_1551828934138.png',
 'start': u'2018-12-12T00:00:00Z',
 'status_message': None,
 'width': None}


### Embedding the screenshots into a Topic Document
Seeq content in Topic documents has a specific format, with custom html attributes.  To fill in the attributes, I'm going to create a Jinja2 template.  Typically these templates are hosted in external html files, but we'll create one from a string for clarity.  In addition to the attributes, we want a link to our image's source data to enable drill-down functionality, so I need to create a link to the source Worksheet.  Note that I create a new UUID for the image when I embed it.  This is for frontend content tracking.

In [47]:
seeq_content_html_tempate = '\
<p>\
<a href="{{worksheet_link}}" class="">\
    <img\
        id="{{image_id}}"\
        data-seeq-content=""\
        class="report-image-border fr-dii specReportSeeqContent fr-draggable"\
        data-seeq-workbookid="{{workbook_id}}"\
        data-seeq-worksheetid="{{worksheet_id}}"\
        data-seeq-workstepid="{{workstep_id}}"\
        data-seeq-datevariableid="{{date_variable_id}}"\
        data-seeq-contentheight="{{content_height}}"\
        data-seeq-contentwidth="{{content_width}}"\
        src="{{screenshot_url}}">\
</a>\
</p>'

#  link creation
protocol = 'http'
host_url = 'localhost'
host_port = 34216

# scorecard link
scorecard_worksheet_link = \
    r'{0}://{1}:{2}/view/worksheet/{3}/{4}?workstepId={5}&;displayRangeStart={6}&;displayRangeEnd={7}&;trendItems='.format(
        protocol, host_url, host_port,
        analysis_output.id,
        worksheet_output.id,
        analysis_workstep_output.id,
        screenshot_date_range['start_iso'],
        screenshot_date_range['end_iso'])

# trend link
trend_worksheet_link = \
    r'{0}://{1}:{2}/view/worksheet/{3}/{4}?workstepId={5}&;displayRangeStart={6}&;displayRangeEnd={7}&;trendItems='.format(
        protocol, host_url, host_port, 
        analysis_output.id,
        worksheet_output.id,
        trend_workstep_output.id,
        screenshot_date_range['start_iso'],
        screenshot_date_range['end_iso'])

jinja_seeq_environment = ji.Environment(loader=ji.BaseLoader).from_string(seeq_content_html_tempate)

scorecard_html = jinja_seeq_environment.render(worksheet_link=scorecard_worksheet_link,
                                               image_id=uuid4(),
                                               workbook_id=analysis_output.id,
                                               worksheet_id=worksheet_output.id,
                                               workstep_id=analysis_workstep_output.id,
                                               date_variable_id=screenshot_date_range['id'],
                                               screenshot_url=metric_screenshot_job_output.screenshot,
                                               content_height=metric_screenshot_job_output.height,
                                               content_width=metric_screenshot_job_output.width)

trend_html = jinja_seeq_environment.render(worksheet_link=trend_worksheet_link,
                                           image_id=uuid4(), workbook_id=analysis_output.id,
                                           worksheet_id=worksheet_output.id,
                                           workstep_id=trend_workstep_output.id,
                                           date_variable_id=screenshot_date_range['id'],
                                           screenshot_url=trend_screenshot_job_output.screenshot,
                                           content_height=trend_height,
                                           content_width=trend_width)

# now to combine them
seeq_html = scorecard_html + trend_html

The final step is to update our Annotation with the html we just created.  Details on these calls can be found in the companion how to _Creating Organizer Topics using the API_.  I will be reusing the `document_output` as well as the `annotation_output` items from that how-to.  These contain the IDs of the Topic document and Annotation that we created using the API.

In [48]:
seeq_data_annotation_input = sdk.AnnotationInputV1()
seeq_data_annotation_input.name = "Unnamed"
seeq_data_annotation_input.type = "Report"
seeq_data_annotation_input.createdById = "Seth Gilchrist"

seeq_data_annotation_interested_input = sdk.AnnotationInterestInputV1()
seeq_data_annotation_interested_input.interest_id = document_output.id
seeq_data_annotation_input.interests = [annotations_interested_input]

seeq_data_annotation_input.document = seeq_html

seeq_data_annotation_update_output = annotations_api.update_annotation(
    id=annotation_output.id, body=seeq_data_annotation_input)
print(seeq_data_annotation_update_output)

{'access_level': u'FULL_CONTROL',
 'backups': [{'backup_date': u'2019-03-05T23:34:00Z',
              'backup_name': u'DocumentBackup\u02c9seth.gilchrist@seeq.com\u02c92019-03-05T23:34:00Z',
              'username': u'seth.gilchrist@seeq.com'},
             {'backup_date': u'2019-03-05T23:32:00Z',
              'backup_name': u'DocumentBackup\u02c9seth.gilchrist@seeq.com\u02c92019-03-05T23:32:00Z',
              'username': u'seth.gilchrist@seeq.com'},
             {'backup_date': u'2019-03-05T22:12:00Z',
              'backup_name': u'DocumentBackup\u02c9seth.gilchrist@seeq.com\u02c92019-03-05T22:12:00Z',
              'username': u'seth.gilchrist@seeq.com'}],
 'created_at': u'2019-03-05T22:12:22.345Z',
 'created_by': {'href': u'/users/7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'id': u'7311CCDB-8572-4BB9-962A-E91D97095B8F',
                'is_archived': False,
                'name': u'Seth Gilchrist',
                'type': u'User'},
 'description': None,
 'discoverabl

You can now go to the Topic, open the document and find the scorecard, with all the Seeq frontend tools at your disposal.

## Taking the Next Step 
Using the same techniques, we can embed all the different kinds of Seeq content in to Topic documents.

To explore different kinds of Seeq content, I encourage you to use the "Inspect" capability of Chrome to examine the html that the frontend uses and creates, and also to take a look at the different data stores in worksteps that you are interested in.  One useful method of understanding the effects of different operations is to use a diff tool (e.g., Meld) to examine changes made to the data store between operations.

Once you have a feel for how the different operations and views manifest in the data store and html, it becomes possible to highly customize your views using the API and create a wide variety of content.

As a final note, I'd like to point out that, as one of our developers, says, "The frontend eats it's own dog food - everything the frontend does is done through the API." So the sky's the limit once you understand how data is stored and objects are represented.

## Contact Us!
If you have questions, comments, or need more help or information, please reach out to your Seeq representative to get a team or 1:1 training on these tools.  We're here to help!